In [1]:
dia = ''

In [2]:
# Parameters
dia = "2024-09-01"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
from scripts.AtualizaUltimosJogos import *
import warnings
warnings.filterwarnings('ignore')

# Pegando os jogos do dia para atualizar
jogos_passados = pd.read_csv(f'jogos_do_dia/{dia}.csv')
dia_jogos = jogos_passados['Date'].iloc[0]
id_jogos = jogos_passados['Fixture ID'].unique().tolist()
print(f'Jogos do dia {dia_jogos}\n')

# Buscando os resultados
results = atualiza_ultimos_jogos(id_jogos = id_jogos)

# Merjando os resultados no dataset de jogos do dia
df_resultados = pd.merge(left=jogos_passados, right=results[['Fixture ID', 'Home_Pts', 'Away_Pts', 'Status']], on='Fixture ID', how='left')
df_resultados['Status'].replace(np.nan, 'Não iniciado', inplace=True)

# Organizando as colunas do df_resultados
df_resultados = df_resultados[['Fixture ID', 'Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', 'Status']]
df_resultados = df_resultados.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
df_resultados.sort_values('Time', inplace=True)
df_resultados.reset_index(inplace=True, drop=True)
# Criando df com jogos que ainda não terminaram
df_restantes = df_resultados[(df_resultados['Status'] != 'Após Tempo Extra') & (df_resultados['Status'] != 'Encerrado') & (df_resultados['Status'] != 'Após Pênaltis')]
df_restantes = df_restantes[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Status']]

# Limpando o df_resultados
df_resultados = df_resultados[(df_resultados['Status'] == 'Após Tempo Extra') | (df_resultados['Status'] == 'Encerrado') | (df_resultados['Status'] == 'Após Pênaltis')]
df_resultados = df_resultados[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A']]

# Exclui os registros que já existem no dataset da temporada
temporada = pd.read_csv('data/Entire_Season.csv')
n_antes = len(temporada)
temporada = pd.concat([temporada, df_resultados], ignore_index=True)
temporada = temporada.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
temporada.reset_index(drop=True, inplace=True)
temporada.to_csv('data/Entire_Season.csv', index=False)

print(f'Dataset de temporada atualizado com {len(temporada) - n_antes} jogos.\n')
print('----------------------------------------------------------\n')
print(f'Atenção! {len(df_restantes)} jogos não finalizados.\n')
display(df_restantes)

Jogos do dia 2024-09-01



  0%|                                                                                           | 0/21 [00:00<?, ?it/s]

  5%|███▉                                                                               | 1/21 [00:01<00:25,  1.28s/it]

 10%|███████▉                                                                           | 2/21 [00:02<00:20,  1.10s/it]

 14%|███████████▊                                                                       | 3/21 [00:03<00:18,  1.05s/it]

...

 95%|██████████████████████████████████████████████████████████████████████████████    | 20/21 [00:18<00:00,  1.08it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:19<00:00,  1.07it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:19<00:00,  1.06it/s]

Dataset de temporada atualizado com 16 jogos.

----------------------------------------------------------

Atenção! 5 jogos não finalizados.



,Date,League,Time,Home,Away,Home_Pts,Away_Pts,Status
0,2024-09-01,Singapura Nbl,01:00,Eng Tat Hornets,Tagawa,37.0,45.0,3º Quarto6'
1,2024-09-01,Singapura Nbl,02:45,SBA,Tong Whye,NaN,NaN,Não iniciado
3,2024-09-01,Singapura Nbl,04:30,SG Basketball,Xin Hua,NaN,NaN,Não iniciado
6,2024-09-01,Singapura Nbl,06:15,Nanyang,Chong Ghee,NaN,NaN,Não iniciado
7,2024-09-01,Singapura Nbl,08:00,Adroit,Siglap,NaN,NaN,Não iniciado
